Test ActionVAE

In [1]:
# prepare data
from data_utils import get_data
dataset_used = 'UTDVideo'
train_data_loader, val_data_loader, __ = get_data(dataset_used, 1)

Train size:  774


In [12]:
from vae import VAE, ActionVAE
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
plt.rcParams['figure.figsize'] = [10, 8]
import torch 

VAEmodel = VAE(latent_dim=512).eval()
VAEmodel_name = 'vae_mark'+str(VAEmodel.mark)+'_UTD'
VAEmodel.load_state_dict(torch.load('models/'+VAEmodel_name+'.pth'))

model = ActionVAE(VAEmodel).eval()
model_name = 'actionvae_mark'+str(model.mark)+'_'+dataset_used

fig, axs = plt.subplots(1, 2)
plt.close()
frames = []
for i, batch in enumerate(val_data_loader):
    vid, label = batch['clip'], batch['label']
    gen = model(vid)[0].squeeze(0)
    vid = vid.squeeze(0)
    frame_num = vid.shape[0]
    for i in range(frame_num):
        frame = vid[i].permute(1, 2, 0)
        im_0 = axs[0].imshow(frame.numpy(), animated=True)

        gen_frame = gen[i].permute(1, 2, 0)
        im_1 = axs[1].imshow(gen_frame.cpu().detach().numpy(), animated=True)
        frames.append([im_0, im_1])
    break
ani = animation.ArtistAnimation(fig, frames, interval=50, blit=True,
                                repeat_delay=1000)

display(HTML(ani.to_html5_video()))